In [ ]:
from os import chdir
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
chdir("/content/drive/MyDrive/Eliott/files/")

Mounted at /content/drive


In [ ]:
# IMPORTS

import numpy as np
import pandas as pd




In [ ]:
# Ouverture

df_train = pd.read_csv('Corona_NLP_train.csv', sep=',', encoding='latin')

df_test = pd.read_csv('Corona_NLP_test.csv', sep=',', encoding='latin')



In [ ]:
df_train.shape

(41157, 6)

In [ ]:
df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
# PREPROCESSING

On vérifie qu'il n'y a pas de null (autre que location) et de ligne dupliquées

In [ ]:
df_train.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [ ]:
df_train = df_train.drop(['Location'], axis=1)
df_test = df_test.drop(['Location'], axis=1)



In [ ]:
df_train['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [ ]:
df_train.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [ ]:
train_df2 = df_train.dropna()
test_df2 = df_test.dropna()



In [ ]:
train_df2.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [ ]:
train_df3 = train_df2.drop_duplicates()
test_df3 = test_df2.drop_duplicates()

In [ ]:
train_df3.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys


In [21]:
!pip install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=4eca8ed29d5789e52f68446d36c93ad2f759132f8d2acd8b8f8be29600dfcbbd
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor


In [22]:
import preprocessor as p


In [23]:
def preprocess_tweet(row):
    text = row['OriginalTweet']
    text = p.clean(text)
    return text

In [24]:
train_df3['OriginalTweet'] = train_df3.apply(preprocess_tweet, axis=1)
test_df3['OriginalTweet'] = test_df3.apply(preprocess_tweet, axis=1)

AttributeError: ignored

In [25]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [26]:
train_df3['OriginalTweet'] = train_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
test_df3['OriginalTweet'] = test_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [27]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,menyrbie phil_gahan chrisitv https t co ifz9f...,Neutral
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,me ready to go at supermarket during the covid...,Extremely Negative


In [28]:
def give_number_to_class(row):
    sent = row['Sentiment']

    if sent=='Extremely Negative':
        return 0
    elif sent=='Negative':
        return 1
    elif sent=='Neutral':
        return 2
    elif sent=='Positive':
        return 3
    else:
        return 4


    

In [29]:
train_df3['Sentiment_Number'] = train_df3.apply(give_number_to_class, axis=1)
test_df3['Sentiment_Number'] = test_df3.apply(give_number_to_class, axis=1)



In [30]:
train_df3.head()

,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment,Sentiment_Number
0,3799,48751,16-03-2020,menyrbie phil_gahan chrisitv https t co ifz9f...,Neutral,2
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive,3
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive,3
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive,3
4,3803,48755,16-03-2020,me ready to go at supermarket during the covid...,Extremely Negative,0


In [31]:
np.unique(train_df3["Sentiment"].values)

array(['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral',
       'Positive'], dtype=object)

In [32]:
X_train = train_df3['OriginalTweet']  
X_test = test_df3['OriginalTweet']  

y_train = train_df3['Sentiment_Number']
y_test = test_df3['Sentiment_Number']

In [33]:
X_train[0]

' menyrbie phil_gahan chrisitv https t co ifz9fan2pa and https t co xx6ghgfzcc and https t co i2nlzdxno8'

In [34]:
df_train.isnull().sum()

UserName         0
ScreenName       0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [35]:
df_train['TweetAt'].values

array(['16-03-2020', '16-03-2020', '16-03-2020', ..., '14-04-2020',
       '14-04-2020', '14-04-2020'], dtype=object)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(41157, 80424)

In [37]:
X_train_tfidf[1]


<1x80424 sparse matrix of type '<class 'numpy.float64'>'
	with 33 stored elements in Compressed Sparse Row format>

In [38]:
tfidf  = TfidfVectorizer()
tfidf.fit_transform(X_train)


model = SGDClassifier()

NameError: ignored

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier


# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC


grid_params = {
  'clf__penalty': ['l2', 'l1', 'elasticnet'],
  'clf__alpha': np.linspace(0.00001, 0.001, 10),  
}



pipeline_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])


clf = GridSearchCV(pipeline_clf, grid_params)
clf.fit(X_train, y_train)


print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)



# Feed the training data through the pipeline
#text_clf.fit(X_train, y_train) 






In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

train_acc = round(accuracy_score(predictions, y_test)*100)
train_acc

In [ ]:
len(y_test)

In [ ]:
print(classification_report(predictions, y_test, target_names=train_df3['Sentiment'].unique()))

In [39]:
X_test[0]

'trending new yorkers encounter empty supermarket shelves pictured wegmans in brooklyn sold out online grocers foodkick maxdelivery as coronavirus fearing shoppers stock up https t co gr76pcrlwh https t co ivmkmsqdt1'

In [40]:
# ACCURACY
c=0
num=len(predictions)
for k in range(num):
    if(predictions[k]==y_test[k]):
        c+=1

print(f"pourcentage de reussite : {(c*100)/num} ")




NameError: ignored

In [ ]:
# L2 ERROR

num=len(predictions)
err_tot = 0
for k in range(num):
    #print(predictions[k])
    if(k<20):
        print(f"prediction {predictions[k]}  , realité : {y_test[k]}")
    err_tot += np.square(predictions[k]-y_test[k])
err_tot/=num
err_tot = np.sqrt(err_tot) 
err_tot